In [5]:
import re

import numpy as np
import pandas as pd
import os
import featuretools as ft
from shl.prepare import normalize_epoch_time, normalize_lat_long
import plotly.express as px
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import DistanceMetric

from shl.features import WifiFeature
import matplotlib
import folium
import branca.colormap as cm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
train_location = normalize_epoch_time(pd.read_parquet('../data/train/Location.parquet'), 'epoch_time')
test_location = normalize_epoch_time(pd.read_parquet('../data/test/Location.parquet'), 'epoch_time')
validate_location = normalize_epoch_time(pd.read_parquet('../data/validate/Location.parquet'), 'epoch_time')
train_label = normalize_epoch_time(pd.read_parquet('../data/train/Label.parquet'), 'epoch_time')
test_label = normalize_epoch_time(pd.read_parquet('../data/test/Label_idx.parquet'), 'epoch_time')
validate_label = normalize_epoch_time(pd.read_parquet('../data/validate/Label.parquet'), 'epoch_time')

In [10]:
feature_points = {
    'London': [
        [51.509865, -0.118092]
    ],
    'Brighton': [
        [50.827778, -0.152778]
    ],
    'parks': [

    ],
    'shopping malls': [

    ],
    'stations': [

    ]
}

def calculate_minimal_distance(data: pd.DataFrame, points):
    dist = DistanceMetric.get_metric('haversine')
    dist_matrix = dist.pairwise(data[['Latitude','Longitude']], points)
    return dist_matrix.min(axis=1)

def create_distance_features(data: pd.DataFrame):
    features = data[['epoch_time_id']]
    for name, points in feature_points.items():
        if len(points) > 0:
            features[f'distance_{name}'] = calculate_minimal_distance(data, points)

    return features

train_distances = create_distance_features(train_location)
display(train_distances)

train_distances.to_parquet('../data/train/features_distances.parquet')
create_distance_features(test_location).to_parquet('../data/test/features_distances.parquet')
create_distance_features(validate_location).to_parquet('../data/validate/features_distances.parquet')

,epoch_time_id,London,Brighton
0,1490431658000,0.665419,0.023635
1,1490431660000,0.665422,0.023632
2,1490431661000,0.665432,0.023621
3,1490431662000,0.665444,0.023613
4,1490431663000,0.665645,0.023557
...,...,...,...
911104,1499268278000,0.663119,0.025111
911105,1499268279000,0.663119,0.025111
911106,1499268280000,0.663119,0.025111
911107,1499268281000,0.663119,0.025111


In [7]:
dist = DistanceMetric.get_metric('haversine')
dist_matrix = dist.pairwise(train_location[['Latitude','Longitude']], feature_points['London'])
# Note that 3959 is the radius of the earth in miles
# df_dist_matrix = (
#     pd.DataFrame(dist_matrix,index=locations_A['city_A'],
#                  columns=locations_B['city_B'])
# )

display(dist_matrix.min(axis=1))

array([0.66541873, 0.66542151, 0.6654316 , ..., 0.66311923, 0.66311923,
       0.66311772])